In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import urllib3

In [2]:
# Importing dataframe
df = pd.read_excel("data\Extract CFNews - Corporate Finance.xlsx")

<>:2: SyntaxWarning: invalid escape sequence '\E'
<>:2: SyntaxWarning: invalid escape sequence '\E'
C:\Users\YoussefKABBAJ\AppData\Local\Temp\ipykernel_28540\1341941611.py:2: SyntaxWarning: invalid escape sequence '\E'
  df = pd.read_excel("data\Extract CFNews - Corporate Finance.xlsx")


In [3]:
df.head(5)

,Identifiant,MainNodeId,Version,Création,Modification,Société cible ou acteur,Siret,Email de la cible,Site internet de la cible,Région,...,Société DD Juridique et Fiscale,Dette Arrangeur,Listing sponsor / Chef de file / Banque présentatrice,Mots clés,Acquéreur Avocat structuration fiscale,Cédant Avocat Structuration fiscale,Cédant Avocat d'affaires Immobilier,URL,Article lié,Date de mise à jour Analyste
0,495207,515967,54,08/08/2024,16/11/2024,AETHER FINANCIAL SERVICES,81147538300022,abernard@aetherfs.com,https://aetherfs.com/,Île-de-France,...,NaN,NaN,NaN,services financiers|capital investissement|con...,NaN,NaN,NaN,https://www.cfnews.net/Annuaires-base-de-deals...,https://www.cfnews.net/L-actualite/Exclusif-CF...,2024-09-05
1,498547,519330,34,19/09/2024,16/11/2024,ODERIS,48971885800058,stephanie.yakhou@oderis.fr,http://www.oderis.fr/,Île-de-France,...,NaN,NaN,NaN,conseil,NaN,NaN,NaN,https://www.cfnews.net/Annuaires-base-de-deals...,https://www.cfnews.net/Les-alertes-de-CFNEWS/O...,2024-09-20
2,474997,495552,40,29/02/2024,26/10/2024,BEX CAPITAL,83825739200038,contact@bexcapital.com,http://www.bexcapital.com/,Région Sud - Provence-Alpes-Côte d'Azur,...,NaN,NaN,NaN,fonds de fonds|secondaire|marché secondaire,NaN,NaN,NaN,https://www.cfnews.net/Annuaires-base-de-deals...,https://www.cfnews.net/L-actualite/LBO/Operati...,2024-07-29
3,465515,486018,83,14/12/2023,09/11/2024,FINANCIÈRE ARBEVEL (VOIR MONTPENSIER ARBEVEL),41468266600048,flhenoret@arbevel.com,https://www.arbevel.com/fr,Île-de-France,...,NaN,NaN,NaN,capital investissement|services financiers|inv...,NaN,NaN,NaN,https://www.cfnews.net/Annuaires-base-de-deals...,https://www.cfnews.net/L-actualite/LBO/Operati...,2024-06-04
4,453216,473647,131,05/09/2023,02/11/2024,NEXTSTAGE AM,44266683000033,info-investor@nextstage.com,https://nextstage-am.com/,Île-de-France,...,NaN,NaN,NaN,capital investissement,NaN,NaN,NaN,https://www.cfnews.net/Annuaires-base-de-deals...,https://www.cfnews.net/L-actualite/LBO/Operati...,2023-10-12


In [4]:
# Disabling warnings about ssl certificated because we will be making requests without ssl (cause some website have expired ssl certificates we need to ignore)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [10]:
def scrape_company_text(url):
    try:
        # Fetch webpage content
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }  # Simulate being a browser to not be blocked by website
        response = requests.get(url, headers=headers, timeout=10, verify=False)

        # Detect encoding if not provided by the server
        response.encoding = (
            "utf-8"  # Make sure all characters such as é à etc are correctly rendered
        )

        response.raise_for_status()  # Raise error for bad responses

        # Parse the HTML
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract title and meta description
        title = soup.title.string if soup.title else ""
        meta_desc = ""
        if soup.find("meta", attrs={"name": "description"}):
            meta_desc = soup.find("meta", attrs={"name": "description"}).get("content", "")

        # Extract text from headers and paragraphs
        headers = [h.get_text(strip=True) for h in soup.find_all(["h1", "h2", "h3"])]
        paragraphs = [p.get_text(strip=True) for p in soup.find_all("p")]

        # Combine all extracted text
        extracted_text = f"Title: {title} \\n Meta Description: {meta_desc} \\n "
        extracted_text += "\\n".join(headers + paragraphs)
        cleaned_text = " ".join(extracted_text.split())
        return cleaned_text

    except Exception as e:
        print(f"Error: {e}")
        return ""

In [6]:
# Example usage
url = "http://www.oderis.fr"
company_text = scrape_company_text(url)
print(company_text)

Title: Oderis - Conseil en création de valeur Meta Description: Cabinet indépendant de conseil financier en transaction et accompagnement opérationnel, pour les fonds d'investissements et dirigeants. Pays/région Pays/région Cabinet indépendanten transaction et accompagnement opérationnel 1erconseil LBOFrance 1erconseil Small CapFrance 2èmeconseil Mid CapFrance 10èmeconseil LBOEurope Oderis portedes valeurs fortes et différenciantessur son marché Entrepreneurship RSE & ESG Business partner Human Focused Nosmétiers Transaction Services Restructuring Transformation Évaluation Tax & Legal ESG 150 Consultants à vos côtés, etheureux de l’être! Oderis enquelques chiffres Collaborateurs Bureaux Associés et directeurs Lignes de service Nos clientsparlent de nous Nosactualités Oderis poursuit sa mue à l’international Oderis poursuit sa mue à l’international Oderis renforce encore son bureau lyonnais Oderis renforce encore son bureau lyonnais Oderis lance son offre ESG avec Radia Benhallam Oderis

In [37]:
df["Site internet de la cible"]

0                                 https://aetherfs.com/
1                                 http://www.oderis.fr/
2                            http://www.bexcapital.com/
3                            https://www.arbevel.com/fr
4                             https://nextstage-am.com/
5                               https://www.123-im.com/
6                                   https://rivalen.fr/
7                             http://www.sorgemeval.fr/
8                             https://www.gocapital.fr/
9                                 http://www.valtus.fr/
10                             http://www.cerea.com/fr/
11                           https://forwardglobal.com/
12                               http://www.finaxim.fr/
13                                http://www.valtus.fr/
14                              https://www.123-im.com/
15                              https://www.swen-cp.fr/
16                        http://www.june-partners.com/
17                                              

In [11]:
df["Homepage Text"] = df["Site internet de la cible"].apply(
    lambda x: scrape_company_text(x) if x is not np.nan else x
)

Error: 502 Server Error: Bad Gateway for url: https://www.cerea.com/fr/


In [8]:
df["Homepage Text"]

0     Title: Aether Financial Services - Leveraging ...
1     Title: Oderis - Conseil en création de valeur ...
2     Title: None Meta Description: The leading buye...
3     Title: ARBEVEL – Une maison de recherche au se...
4     Title: Accueil - NextStage AM, Spécialiste du ...
5     Title: 123 IM - Solutions de diversification d...
6     Title: Rivalen, collectif d’industries françai...
7     Title: Sorgem Évaluation : Évaluation, content...
8     Title: Go capital - Leader de l’investissement...
9     Title: Leader Européen du Management de Transi...
10    Title: Cerea Partners - L'Investisseur Partena...
11    Title: Forward Global - Forward Global Meta De...
12    Title: Cabinet directeurs externalisés - Finax...
13    Title: Leader Européen du Management de Transi...
14    Title: 123 IM - Solutions de diversification d...
15    Title: SWEN CP | Investisseur responsable en n...
16    Title: Site déconnecté Meta Description: Site ...
17                                              

In [13]:
df.to_excel("data/resultats.xlsx")